#Using Options

Please complete this tutorial to get an overview of options and an implementation of SMDP Q-Learning and Intra-Option Q-Learning.


### References:

 [Recent Advances in Hierarchical Reinforcement
Learning](https://people.cs.umass.edu/~mahadeva/papers/hrl.pdf) is a strong recommendation for topics in HRL that was covered in class. Watch Prof. Ravi's lectures on moodle or nptel for further understanding the core concepts. Contact the TAs for further resources if needed. 


In [ ]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
import random
import gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML

In [ ]:
'''
The environment used here is extremely similar to the openai gym ones.
At first glance it might look slightly different. 
The usual commands we use for our experiments are added to this cell to aid you
work using this environment.
'''

#Setting up the environment
env = gym.make('Taxi-v3')
state= env.reset()

# Print dimensions of state and action space
print("State space: {}".format(env.observation_space))
print("Action space: {}".format(env.action_space))

# Sample random action
action = env.action_space.sample(env.action_mask(state))
next_state, reward, done, _= env.step(action)

# Print output
print("State: {}".format(state))
print("Action: {}".format(action))
print("Action mask: {}".format(env.action_mask(state)))
print("Reward: {}".format(reward))


State space: Discrete(500)
Action space: Discrete(6)
State: 107
Action: 2
Action mask: [1 1 1 0 0 0]
Reward: -1


/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
!pip show gym

Name: gym
Version: 0.25.2
Summary: Gym: A universal API for reinforcement learning environments
Home-page: https://www.gymlibrary.ml/
Author: Gym Community
Author-email: jkterry@umd.edu
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: cloudpickle, gym-notices, importlib-metadata, numpy
Required-by: dopamine-rl


In [ ]:
def stateData(state):
  picdrop = state%20
  pos = state//20
  x = pos//5
  y = pos%5
  drop = picdrop%4
  passenger = picdrop//4
  return x,y,drop,passenger

New set of options


In [ ]:
goal = {0:[0,0],1:[0,4],2:[4,0],3:[4,3]}

def gotoR_PU(env,state):
    optpolicy = np.array([[4,3,0,0,0],
                 [1,1,0,3,3],
                 [1,1,3,3,3],
                 [1,1,1,1,1],
                 [1,1,1,1,1]
                 ])
    optdone = False
    
    x, y, drop, pas = stateData(state) 
    if ([x,y]==goal[0]):
        optdone = True
    
    optact = optpolicy[x][y]
    return [optact,optdone]
    
def gotoG_PU(env,state):
    optpolicy = np.array([[0,0,2,2,4],
                 [2,0,2,2,1],
                 [2,2,2,2,1],
                 [1,1,1,1,1],
                 [1,1,1,1,1]
                 ])
    optdone = False
    
    x, y, drop, pas = stateData(state) 
    if ([x,y]==goal[1]):
        optdone = True
    optact = optpolicy[x,y]
    return [optact,optdone]

def gotoY_PU(env,state):
  optpolicy = np.array([[0,0,0,0,0],
                          [0,0,0,0,0],
                          [0,3,3,3,3],
                          [0,1,1,1,3],
                          [4,1,3,1,3]
                          ])
  optdone = False
  
  x, y, drop, pas = stateData(state) 
  if ([x,y]==goal[2]):
      optdone = True
  optact = optpolicy[x,y]
  return [optact,optdone] 

def gotoB_PU(env,state):
  optpolicy = np.array([[0,0,0,0,3],
                          [0,0,0,0,3],
                          [2,2,2,0,3],
                          [1,1,1,0,3],
                          [1,1,1,4,3]
                          ])
  optdone = False
  
  x, y, drop, pas = stateData(state) 
  if ([x,y]==goal[3]):
      optdone = True
  optact = optpolicy[x,y]
  return [optact,optdone] 
    
def gotoR_DD(env,state):
    optpolicy = np.array([[5,3,0,0,0],
                 [1,1,0,3,3],
                 [1,1,3,3,3],
                 [1,1,1,1,1],
                 [1,1,1,1,1]
                 ])
    optdone = False
    
    x, y, drop, pas = stateData(state) 
    if ([x,y]==goal[0]):
        optdone = True
    
    optact = optpolicy[x][y]
    return [optact,optdone]
    
def gotoG_DD(env,state):
    optpolicy = np.array([[0,0,2,2,5],
                 [2,0,2,2,1],
                 [2,2,2,2,1],
                 [1,1,1,1,1],
                 [1,1,1,1,1]
                 ])
    optdone = False
    
    x, y, drop, pas = stateData(state) 
    if ([x,y]==goal[1]):
        optdone = True
    optact = optpolicy[x,y]
    return [optact,optdone]

def gotoY_DD(env,state):
  optpolicy = np.array([[0,0,0,0,0],
                          [0,0,0,0,0],
                          [0,3,3,3,3],
                          [0,1,1,1,3],
                          [5,1,3,1,3]
                          ])
  optdone = False
  
  x, y, drop, pas = stateData(state) 
  if ([x,y]==goal[2]):
      optdone = True
  optact = optpolicy[x,y]
  return [optact,optdone] 

def gotoB_DD(env,state):
  optpolicy = np.array([[0,0,0,0,3],
                          [0,0,0,0,3],
                          [2,2,2,0,3],
                          [1,1,1,0,3],
                          [1,1,1,5,3]
                          ])
  optdone = False
  
  x, y, drop, pas = stateData(state) 
  if ([x,y]==goal[3]):
      optdone = True
  optact = optpolicy[x,y]
  return [optact,optdone]   
  '''
Now the new action space will contain
Primitive Actions: ["up", "right", "down", "left"]
Options: ["Away","Close"]
Total Actions :["up", "right", "down", "left", "Away", "Close"]
Corresponding to [0,1,2,3,4,5]
'''

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# code to get the specific pos of taxi, 

#### Options
We custom define very simple options here. They might not be the logical options for this settings deliberately chosen to visualise the Q Table better.


# Task 1
Complete the code cell below


In [ ]:



# TODO: epsilon-greedy action selection function
def egreedy_policy(q_values,state,epsilon):
    decision = np.random.choice(["explore", "exploit"], p = [epsilon, 1- epsilon])
    if decision == "explore":
        return np.random.randint(q_values.shape[1])
    else: 
        return np.argmax(q_values[state])
    # if np.random.rand() < epsilon :
    #     return np.random.randint(q_values.shape[1])
    # else:
    #     return np.argmax(q_values[state])

# Task 2
Below is an incomplete code cell with the flow of SMDP Q-Learning. Complete the cell and train the agent using SMDP Q-Learning algorithm.
Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)

In [ ]:
#### SMDP Q-Learning 
#Q-Table: (States x Actions) === (env.ns(48) x total actions(6))
q_values_SMDP = np.zeros((500,14))


#Update_Frequency Data structure? Check TODO 4
SMDP_freq = np.zeros((500, 14))

optDict = {
    6: gotoR_PU,
    7: gotoG_PU,
    8: gotoY_PU,
    9: gotoB_PU,
    10: gotoR_DD,
    11: gotoG_DD,
    12: gotoY_DD,
    13: gotoB_PU

}
# Add parameters you might need here
gamma = 0.9
alpha = 0.8

# Iterate over 1000 episodes
steps = np.zeros(1000)
rewards = np.zeros(1000)
for i in range(1000):
    print("episode =", i)
    state = env.reset()    
    done = False

    # While episode is not over
    t = 1
    while not done:
        
        state_ = state
        # Choose action   
        x, y, drop, pas = stateData(state)

        # if pas != 4 and [x,y] == goal[pas]:
        #   action = 4
        # else:
        action = egreedy_policy(q_values_SMDP, state, epsilon=0.1)
        if pas == 4 :
          print("succesfully picked up passenger")

        print(f"state = {stateData(state)}")
        print("action =", action)
        # Checking if primitive action

        if action < 6:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done,_= env.step(action)
            t += 1
            q_values_SMDP[state][action] = q_values_SMDP[state][action] + alpha * (reward + gamma * np.max(q_values_SMDP[next_state]) - q_values_SMDP[state][action])
            SMDP_freq[state][action] += 1
            state = next_state
        # Checking if action chosen is an option
        
        else : 
            reward_bar = 0
            tau = 0
            optdone = False
            
            while (optdone == False):
                
                # Think about what this function might do?
                optact,optdone = optDict[action](env,state) 
                
                print("inside option action = ", optact)
                next_state, reward, done,_ = env.step(optact)
                t += 1
                print(f"state = {stateData(next_state)}")
                tau += 1
                # Is this formulation right? What is this term?
                # Yes the formulation is correct and we are finding the cumulative reward for the course of our option
                reward_bar = gamma*reward_bar + reward
            
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update? 
                if optdone or done:  
                    q_values_SMDP[state_][action] += alpha * (reward_bar + (gamma ** tau) * np.max(q_values_SMDP[next_state]) - q_values_SMDP[state_][action])
                    SMDP_freq[state][action] += 1
                state = next_state
            
    if done :
      print("succefully dropped off paasenger in steps = ", t)
      steps[i] = t
           
        


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
succesfully picked up passenger
state = (0, 4, 3, 4)
action = 0
succesfully picked up passenger
state = (1, 4, 3, 4)
action = 3
succesfully picked up passenger
state = (1, 3, 3, 4)
action = 9
inside option action =  0
state = (2, 3, 3, 4)
inside option action =  0
state = (3, 3, 3, 4)
inside option action =  0
state = (4, 3, 3, 4)
inside option action =  4
state = (4, 3, 3, 4)
succesfully picked up passenger
state = (4, 3, 3, 4)
action = 5
succefully dropped off paasenger in steps =  9
episode = 886
state = (2, 3, 0, 1)
action = 7
inside option action =  2
state = (2, 4, 0, 1)
inside option action =  1
state = (1, 4, 0, 1)
inside option action =  1
state = (0, 4, 0, 1)
inside option action =  4
state = (0, 4, 0, 4)
succesfully picked up passenger
state = (0, 4, 0, 4)
action = 10
inside option action =  0
state = (1, 4, 0, 4)
inside option action =  3
state = (1, 3, 0, 4)
inside option action =  3
state = (1, 2, 0, 4)
inside option acti

In [ ]:
np.sum(steps < 201)

979

In [ ]:
np.sum(steps < 20)

533

In [ ]:
steps


array([202., 201., 208., 201., 201., 205., 207.,  88., 201., 201., 201.,
       201., 201., 206., 206., 106.,  77., 201.,  56.,  78., 152., 201.,
       153.,  40., 205.,  62.,  61., 201.,  78.,  43.,  91.,  55.,  42.,
        68.,  98.,  50.,  31., 157., 126., 110.,  30., 201., 143.,  56.,
       201., 201.,  44., 159., 204.,  43.,  49.,  29.,  15., 117.,  65.,
        58., 134.,  33., 100.,  93.,  11.,  82.,  48.,  36.,  93.,  63.,
       143.,  77., 201.,  25.,  27.,  51.,  12.,  36.,  73.,  29.,  33.,
        90., 201.,  59.,  54.,  46.,  17.,   8.,  36.,  64., 201.,  40.,
        75.,  37.,  29., 205., 201.,  17.,  17., 139., 111.,  55.,  54.,
       123., 100.,  87., 156.,  36.,  94.,  92.,  14., 201.,  86.,  20.,
        28., 128.,  84.,  63., 201., 208., 143., 110.,  24.,  13., 101.,
        23.,  57.,  34.,  92., 175.,  41.,  33.,  31.,  44., 199., 201.,
        52., 115.,  47.,  22.,  22.,  39.,  44.,  21.,  80.,  60.,  56.,
        46.,  18.,  38.,  52.,  29.,  19.,  86.,  7

Things to do:

1. run the whole experiment for 10 no. of times and find the average rewards and the avg no. of steps to complete the episode
2. do the above for atleast 4 hyperparameter
3. repeat the above for smdp and io learning

# IO 



In [ ]:
#### Intra-Option Q-Learning 
#### SMDP Q-Learning 
q_values_IO = np.zeros((500,14)) # for intra option
IO_freq = q_values_SMDP = np.zeros((500,14))

# Add parameters you might need here
gamma = 0.9
alpha = 0.8
steps = np.zeros(1000)
# Iterate over 1000 episodes
for i in range(1000):
    print(f"\r{i}",end="")
    state = env.reset()    
    done = False

    t = 1
    # While episode is not over
    while not done:
        
        x, y, drop, pas = stateData(state)
        # Choose action        
        action = egreedy_policy(q_values_IO, state, epsilon=0.1)
        if pas == 4 :
          print("succesfully picked up passenger")

        print(f"state = {stateData(state)}")
        print("action =", action)
        # Checking if primitive action
        if action < 6:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done,_ = env.step(action)
            t += 1
            q_values_IO[state][action] += alpha * (reward + gamma * np.max(q_values_IO[next_state]) - q_values_IO[state][action])
            IO_freq[state][action] += 1
            state = next_state
        # Checking if action chosen is an option
        
        else: # action => Away option
            
            optdone = False
            while (optdone == False and not done):
                
                # Think about what this function might do?
                optact,optdone = optDict[action](env,state) 
                next_state, reward, done,_ = env.step(optact)
                t += 1
                print("inside option action = ", optact)
                print(f"state = {stateData(next_state)}")
                # Complete IO Q-Learning Update
                # update for state and primitive action
                q_values_IO[state][optact] += alpha * (reward + gamma * np.max(q_values_IO[next_state]) - q_values_IO[state][optact])
                IO_freq[state][optact] +=1

                # for other options with same policy for that state also update
                for opt in range(6, 10):
                    if opt != action:  # check all other options
                      optact_, optdone_ = optDict[opt](env,state) # get the action the option would take
                      if optact == optact_:  # if the action taken is same the action of the taken option then we can update
                        q_values_IO[state][opt] += alpha * (reward + gamma *((1-optdone_) * q_values_IO[next_state][opt] + optdone_ * np.max(q_values_IO[next_state]) ) - q_values_IO[state][opt])
                        IO_freq[state][opt] += 1

                # update the option q value
                q_values_IO[state][action] += alpha * (reward + gamma *((1-optdone) * q_values_IO[next_state][action] + optdone * np.max(q_values_IO[next_state]) ) - q_values_IO[state][action])
                IO_freq[state][action] += 1

                # since we know the policy of the other option will never take the same action, we need not update it
                state = next_state
           
    
    print("succefully dropped off paasenger in steps = ", t)
    steps[i] = t    

Streaming output truncated to the last 5000 lines.
state = (3, 0, 3, 2)
action = 0
state = (4, 0, 3, 2)
action = 4
succesfully picked up passenger
state = (4, 0, 3, 4)
action = 1
succesfully picked up passenger
state = (3, 0, 3, 4)
action = 1
succesfully picked up passenger
state = (2, 0, 3, 4)
action = 2
succesfully picked up passenger
state = (2, 1, 3, 4)
action = 2
succesfully picked up passenger
state = (2, 2, 3, 4)
action = 2
succesfully picked up passenger
state = (2, 3, 3, 4)
action = 0
succesfully picked up passenger
state = (3, 3, 3, 4)
action = 5
succesfully picked up passenger
state = (3, 3, 3, 4)
action = 0
succesfully picked up passenger
state = (4, 3, 3, 4)
action = 5
succefully dropped off paasenger in steps =  14
895state = (0, 4, 3, 2)
action = 0
state = (1, 4, 3, 2)
action = 3
state = (1, 3, 3, 2)
action = 0
state = (2, 3, 3, 2)
action = 3
state = (2, 2, 3, 2)
action = 3
state = (2, 1, 3, 2)
action = 3
state = (2, 0, 3, 2)
action = 0
state = (3, 0, 3, 2)
action = 0
st

In [ ]:
np.sum(steps < 201)

961

In [ ]:
np.sum(steps < 20)

522

In [ ]:
steps

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([201., 201., 201., 131., 151., 201., 201., 101., 201., 201., 201.,
       139., 201., 201., 177., 168., 201., 134., 189., 180., 201., 201.,
       201., 201., 179., 201., 201., 201., 201., 201., 179., 157., 119.,
       201., 201., 142., 154., 103., 201., 201.,  79., 144.,  87., 119.,
       201., 174.,  58.,  94., 201., 201., 201., 201.,  48.,  25., 201.,
       201., 175.,  58.,  58., 201.,  72., 201.,  79., 201.,  91., 100.,
       201.,  53.,  44., 201., 201., 170., 201.,  45., 201., 171., 201.,
        76.,  87.,  90., 157., 167.,  93.,  32.,  34., 121.,  46.,  24.,
        72.,  89.,  87., 118.,  41., 146.,  48.,  71., 201., 201.,  26.,
        61., 121., 201., 189.,  98.,  30.,  65., 165.,  56., 201., 201.,
       201.,  45., 117., 121., 199., 101., 201.,  54., 130., 123.,  19.,
       130.,  41., 101.,  62.,  30.,   9.,  65.,  20.,  98.,   8.,  80.,
        59.,  50.,  44.,  52.,  96.,  12.,  14.,  31.,  45., 100.,  36.,
       140.,  21.,  19.,  53.,  53.,  20.,  24., 20

# Task 4
Compare the two Q-Tables and Update Frequencies and provide comments.

In [ ]:
policyIO = np.argmax(q_values_SMDP, axis = 1).reshape(4, 12)
policyIO

array([[3, 0, 3, 3, 0, 1, 1, 3, 3, 3, 4, 3],
       [3, 2, 0, 0, 3, 1, 0, 0, 5, 3, 3, 4],
       [2, 3, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0],
       [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Use this text cell for your comments - Task 4
